In [2]:
import pyigtl
import numpy as np
from YOLOv5.model import YOLOv5
import torch

weights = 'D:/GitRepos/aigt-LIVE/UltrasoundObjectDetection/YOLOv5/weights/lung_us_pretrained.pt'
target_size = 512
line_thickness = 2
model = None
host = "127.0.0.1"
port = 18944
input_device_name = 'Image_Reference'
output_device_name = 'Inference'
confidence_threshold = 0.5

client = pyigtl.OpenIGTLinkClient(host=host, port=port)

def preprocess_epiphan_image(image):
    image = np.rot90(np.transpose(image, (1,2,0)), 2)
    if image.shape[2] == 1:
        image = np.concatenate([image, image, image], axis=2)
    return image

while True:
    #print('Waiting for message...')
    message = client.wait_for_message(input_device_name, timeout=-1)

    if isinstance(message, pyigtl.ImageMessage):
        if model is None:
            input_size = message.image.shape[1:3]
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = YOLOv5(weights=weights,
                            device=device,
                            line_thickness=line_thickness,
                            input_size=input_size,
                            target_size=target_size)

        image = preprocess_epiphan_image(message.image)

        prediction = model.predict(image, confidence_threshold)
        image_message = pyigtl.ImageMessage(np.flip(np.flip(prediction, axis=1), axis=2), device_name=output_device_name)
        client.send_message(image_message, wait=True)
    else:
        print(f'Unexpected message format. Message:\n{message}')
        



Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs


KeyboardInterrupt: 

Traceback (most recent call last):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 332, in _client_thread_function
    while self._receive_message_from_socket(self.socket):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 114, in _receive_message_from_socket
    header += ssocket.recv(MessageBase.IGTL_HEADER_SIZE - received_header_size)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:root:Error while receiving data: [WinError 10054] An existing connection was forcibly closed by the remote host
Traceback (most recent call last):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 332, in _client_thread_function
    while self._receive_message_from_socket(self.socket):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 114, in _receive_message_from_socket
    header +=

In [1]:
'''
Same thing, but threaded. Input and output ports are separated to allow parallel processing.
'''
import pyigtl
import numpy as np
from threading import Thread
from YOLOv5.model import YOLOv5
import torch
from queue import SimpleQueue
from threading import Thread


weights = 'D:/GitRepos/aigt-LIVE/UltrasoundObjectDetection/YOLOv5/weights/lung_us_pretrained.pt'
target_size = 512
line_thickness = 2
model = None
host = "127.0.0.1"
input_port = 18944
output_port = 18945
confidence_threshold = 0.5
input_queue = SimpleQueue()
output_queue = SimpleQueue()
input_device = "Image_Reference"
output_device = "Inference"
input_client = pyigtl.OpenIGTLinkClient(host=host, port=input_port)
output_client = pyigtl.OpenIGTLinkClient(host=host, port=output_port)


input_size = (800,600)  #message.image.shape[1:3]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = YOLOv5(weights=weights,
                device=device,
                line_thickness=line_thickness,
                input_size=input_size,
                target_size=target_size)


def read_message():
    while True:
        message = input_client.wait_for_message(device_name=input_device, timeout=-1)
        input_queue.put(message.image)

def send_message():
    while True:
        message = pyigtl.ImageMessage(output_queue.get(), device_name=output_device)
        output_client.send_message(message, wait=True)

def process_message():
    while True:
        image = np.rot90(np.transpose(input_queue.get(), (1,2,0)), 2)
        prediction = model.predict(image, confidence_threshold)
        output_queue.put(np.flip(np.flip(prediction, axis=1), axis=2))

read_message_thread = Thread(target=read_message, name='read_message_thread')
process_message_thread = Thread(target=process_message, name='process_message_thread')
send_message_thread = Thread(target=send_message, name='send_message_thread')

read_message_thread.start()
process_message_thread.start()
send_message_thread.start()

read_message_thread.join()
process_message_thread.join()
send_message_thread.join()

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
Traceback (most recent call last):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 332, in _client_thread_function
    while self._receive_message_from_socket(self.socket):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 114, in _receive_message_from_socket
    header += ssocket.recv(MessageBase.IGTL_HEADER_SIZE - received_header_size)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
ERROR:root:Error while receiving data: [WinError 10054] An existing connection was forcibly closed by the remote host
Traceback (most recent call last):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\lib\site-packages\pyigtl\comm.py", line 332, in _client_thread_function
    while self._receive_message_from_socket(self.socket):
  File "c:\Users\Robert\anaconda3\envs\pytorch_lab\